# Import

In [ ]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, Subset

import torch.nn.functional as F
from torch import nn, optim

from sklearn.metrics import log_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from utils.utils import set_seed
from datasets import get_dataset
from datasets.transforms import load_transforms_from_yaml
from models import get_model

Using device: cuda


# Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 64,
    'EPOCHS': 50,
    'LEARNING_RATE': 1e-4,
    'SEED' : 42,
    'DATASET' : 'FastImageDataset',
    'MODEL': 'Resnet50HeadArc',
}

DatasetClass = get_dataset(CFG['DATASET'])
ModelClass = get_model(CFG['MODEL'])

# Fixed RandomSeed

In [3]:
set_seed(CFG['SEED'])

seed 고정 완료!


# Data Load

In [4]:
data_path = './data'
train_root = f'{data_path}/train'
test_root = f'{data_path}/test'

In [ ]:
train_transform, val_transform = load_transforms_from_yaml('./config/transforms_config.yaml')

In [ ]:
# 전체 데이터셋 로드
full_dataset = DatasetClass(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]

class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(DatasetClass(train_root, transform=train_transform), train_idx)
val_dataset = Subset(DatasetClass(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=os.cpu_count() // 2, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=os.cpu_count() // 2, pin_memory=True, persistent_workers=True)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


# Train/ Validation

In [7]:
model = ModelClass(num_classes=len(class_names)).to(device)
best_logloss = float('inf')

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])

# 학습 및 검증 루프
for epoch in range(CFG['EPOCHS']):
    # Train
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        if CFG["MODEL"] != "Resnet50HeadArc":
            outputs = model(images)  # logits
        else:
            outputs = model(images, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
            images, labels = images.to(device), labels.to(device)
            if CFG["MODEL"] != "Resnet50HeadArc":
                outputs = model(images)
            else:
                outputs = model(images, labels)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))

    # 결과 출력
    print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.4f}%")

    # Best model 저장
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        torch.save(model.state_dict(), f'best_model.pth')
        print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.4f})")

[Epoch 1/50] Validation: 100%|██████████| 104/104 [00:07<00:00, 13.64it/s]


Train Loss : 19.6089 || Valid Loss : 18.2073 | Valid Accuracy : 0.0000%
📦 Best model saved at epoch 1 (logloss: 18.2071)


[Epoch 2/50] Validation: 100%|██████████| 104/104 [00:07<00:00, 14.52it/s]


Train Loss : 16.6134 || Valid Loss : 15.8032 | Valid Accuracy : 0.0000%
📦 Best model saved at epoch 2 (logloss: 15.8016)


[Epoch 3/50] Validation: 100%|██████████| 104/104 [00:07<00:00, 14.65it/s]


...

Train Loss : 0.0243 || Valid Loss : 3.1419 | Valid Accuracy : 73.3705%


[Epoch 49/50] Validation: 100%|██████████| 104/104 [00:07<00:00, 14.74it/s]


Train Loss : 0.0216 || Valid Loss : 3.1351 | Valid Accuracy : 74.2758%


[Epoch 50/50] Validation: 100%|██████████| 104/104 [00:07<00:00, 14.56it/s]


Train Loss : 0.0156 || Valid Loss : 3.0713 | Valid Accuracy : 74.5323%
📦 Best model saved at epoch 50 (logloss: 3.0590)


# Inference

In [8]:
test_dataset = DatasetClass(test_root, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [9]:
# 저장된 모델 로드
model = ModelClass(num_classes=len(class_names))
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

# 추론
model.eval()
results = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        # 각 배치의 확률을 리스트로 변환
        for prob in probs.cpu():  # prob: (num_classes,)
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

# Submission

In [10]:
submission = pd.read_csv(f'{data_path}/sample_submission.csv', encoding='utf-8-sig')

# 'ID' 컬럼을 제외한 클래스 컬럼 정렬
class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')